In [ ]:
!pip install nibiru==0.5.0a2 --quiet

> ⚠ **Warning** : Once the above command ended, please restart the python kernel using `⌘/Ctrl + M .`

# Introduction to py-sdk

The python sdk allows you to create queries and transaction to a Nibiru chain.

It allows to interact with all of the modules and can be leveraged to automate trading strategies or monitor them.

This notebook will guide you on opening and closing positions.

In [ ]:
import os
import json

import nibiru
import nibiru.msg

In [ ]:
# We set up the environment variables to connect to devnet
envs = {
  "HOST" : "34.130.24.87",
  "LCD_ENDPOINT" : "http://34.130.24.87:1317",
  "GRPC_ENDPOINT" : "34.130.24.87:9090",
  "TENDERMINT_RPC_ENDPOINT" : "http://34.130.24.87:26657",
  "WEBSOCKET_ENDPOINT" : "ws://34.130.24.87:26657/websocket",
  "CHAIN_ID" : "nibiru-localnet-0",
}

os.environ.update(envs)

In [ ]:
tx_config = nibiru.TxConfig(tx_type=nibiru.common.TxType.BLOCK)
agent = (
    nibiru.Sdk
    .authorize()
    .with_network(nibiru.Network.customnet(), insecure=True)
    .with_config(tx_config)
)

Our trader don't have any fund, so if we try to open a position it will fail saying that the address does not exist (never registered with the auth module of the chain).

Let's use the validator funds to give some unusd and unibi to the trader

In [ ]:
whale_mnemonic = "guard cream sadness conduct invite crumble clock pudding hole grit liar hotel maid produce squeeze return argue turtle know drive eight casino maze host"

validator = (
    nibiru.Sdk
    .authorize(whale_mnemonic)
    .with_network(nibiru.Network.customnet(), insecure=True)
    .with_config(tx_config)
)

In [ ]:
_ = validator.tx.execute_msgs(
    nibiru.msg.MsgSend(
        from_address=validator.address,
        to_address=agent.address,
        coins=[nibiru.Coin(10000, "unibi"), nibiru.Coin(10000, "unusd")]
    )
)

We can check the new balance of the trader:

In [ ]:
agent.query.get_bank_balances(agent.address)

Now with those fresh token, we can open a long position

In [ ]:
_ = agent.tx.execute_msgs(
    nibiru.msg.MsgOpenPosition(
        sender=agent.address,
        token_pair="ubtc:unusd",
        side=nibiru.common.Side.BUY,
        quote_asset_amount=10,
        leverage=10,
        base_asset_amount_limit=0,
    )
)

In [ ]:
agent.query.perp.position(trader=agent.address, token_pair="ubtc:unusd")